<a href="https://colab.research.google.com/github/mueller-florian-rh/menu-extraction/blob/main/cater_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf langchain_openai langchain requests

In [ ]:
import csv
import getpass
import os
import pymupdf
import requests
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from urllib.parse import urlparse

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
prompt_request = requests.get("https://raw.githubusercontent.com/mueller-florian-rh/menu-extraction/refs/heads/main/prompt_speisekarten_extraction.txt")
with open("prompt_speisekarten_extraction.txt", "w", encoding="utf-8") as f:
  f.write(prompt_request.text)
  f.close()
prompt = PromptTemplate(
    template=open("prompt_speisekarten_extraction.txt", encoding="utf-8").read(),
    input_variables=["full_text"]
)

chain = prompt | llm

print("Enter URL:")
url = input()
path = urlparse(url).path
filename = os.path.basename(path)
name_without_ext = os.path.splitext(filename)[0]
tmp_pdf = f"temp_{name_without_ext}.pdf"
r = requests.get(url)
with open(tmp_pdf, "wb") as f:
  f.write(r.content)
  f.close()

doc = pymupdf.open(tmp_pdf)
full_text = "\n".join([page.get_text() for page in doc])

result = chain.invoke({"pdf_text" : full_text})

tmp_csv = f"output_{name_without_ext}.csv"
with open(tmp_csv, "w", encoding="utf-8", newline="") as f:
  writer = csv.writer(f, delimiter=";")
  writer.writerow(["Kategorie", "Speise", "Beschreibung", "Confidence"])
  for line in result.content.strip().split("\n"):
    parts = line.split(";")
    if len(parts) == 4:
      writer.writerow(parts)

os.remove(tmp_pdf)